**Лабораторная №6**

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.

Постройте модель с максимально большим значением accuracy. Нужно довести долю правильных ответов по крайней мере до 0.75. Необходимо применить ансамблевые методы для повышения качества прогноза и подобрать хотя бы для одного из них гиперпараметры. Проверьте accuracy на тестовой выборке самостоятельно.

Описание данных:
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

•	сalls — количество звонков,

•	minutes — суммарная длительность звонков в минутах,

•	messages — количество sms-сообщений,

•	mb_used — израсходованный интернет-трафик в Мб,

•	is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


In [160]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import BaggingClassifier, StackingClassifier, AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [161]:
df = pd.read_csv('users_behavior.csv')
df.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [162]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [163]:
target = df['is_ultra']
df = df.drop(['is_ultra'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.3, random_state=12)

In [164]:
lr = LogisticRegression(solver='lbfgs', max_iter=3000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=3000)

In [165]:
print(lr.score(X_test, y_test))

0.7253886010362695


**Stacking (стекинг)**

In [167]:
estimators = [('rf', RandomForestClassifier(n_estimators=10, random_state=42)),('svr', make_pipeline(StandardScaler(), LinearSVC(dual="auto", random_state=42)))]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

In [168]:
clf.fit(X_train, y_train).score(X_test, y_test)

0.7823834196891192

**Bagging (бэггинг)**

In [170]:
modelClf = BaggingClassifier(estimator=LogisticRegression(), n_estimators=12, random_state=12)

In [171]:
modelClf.fit(X_train, y_train)

BaggingClassifier(estimator=LogisticRegression(), n_estimators=12,
                  random_state=12)

In [172]:
print(modelClf.score(X_train, y_train))

0.7594486438417074


**Random Forest (случайный лес)**

In [174]:
clf = RandomForestClassifier(n_estimators=20, max_depth=6,
                             min_samples_split=2, random_state=0)

In [175]:
scores = cross_val_score(clf, X_train, y_train, cv=3)

In [176]:
scores.mean()

0.8096899569796766

In [177]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=6, n_estimators=20, random_state=0)

In [178]:
print(clf.score(X_train, y_train))

0.840373499333037


Подбор гиперпараметров для случайного леса (через цикл и кросс-валидацию)

In [180]:
best_score = 0
best_params = None

for max_depth in range(2, 10):
    for n_estimators in range(10, 31, 10):
        model_rfr = RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, random_state=12345)
        score = cross_val_score(model_rfr, X_train, y_train, cv=3, n_jobs=-1).mean()
        if score > best_score:
            best_score = score
            best_params = {'max_depth': max_depth, 'n_estimators': n_estimators}

print('Лучшее значение accuracy для случайного леса: {} при значениях гиперпараметров: {}'.format(best_score, best_params))

Лучшее значение accuracy для случайного леса: 0.8172490728378579 при значениях гиперпараметров: {'max_depth': 9, 'n_estimators': 20}


**Boosting (бустинг)**

**Adaboost (адаптивный бустинг)**

In [183]:
modelClf = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, random_state=12)

In [184]:
modelClf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2),
                   n_estimators=100, random_state=12)

In [185]:
print(modelClf.score(X_test, y_test))

0.7637305699481866


**Gradient Boosting (Градиентный бустинг)**

In [187]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                 max_depth=1, random_state=0)

In [188]:
clf.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)

In [189]:
print(clf.score(X_test, y_test))

0.7792746113989637


**Voting (простое усреднение)**

In [191]:
decisiontree = DecisionTreeClassifier(max_depth=2)

In [192]:
forest = RandomForestClassifier(n_estimators=20, max_depth=6,
                             min_samples_split=2, random_state=0)

In [193]:
ensemble=VotingClassifier(estimators=[('Decision Tree', decisiontree), ('Random Forest', forest)],
                       voting='soft', weights=[1,1]).fit(X_train, y_train)

In [194]:
print(ensemble.score(X_test, y_test))

0.7803108808290156
